## Linear Regression with PySpark - Part 2

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lin_reg").getOrCreate()

25/07/14 09:01:06 WARN Utils: Your hostname, aditya-HP-Laptop-15s-eq1xxx resolves to a loopback address: 127.0.1.1; using 10.200.82.42 instead (on interface wlo1)
25/07/14 09:01:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/14 09:01:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/14 09:01:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/14 09:01:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
df = spark.read.csv("ecommerce.csv", inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
df.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

### Setup DataFrame for ML

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [23]:
coef_var = ['Avg Session Length', 'Time on App', 'Length of Membership']
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Length of Membership'], 
                           outputCol='features')

In [24]:
output = assembler.transform(df)

In [25]:
final_df = output.select("features", "Yearly Amount Spent")
final_df.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [26]:
train_data, test_data = final_df.randomSplit([0.7, 0.3])

In [27]:
train_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.3931845423455...|  319.9288698031936|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
|[30.8162006488763...|   266.086340948469|
|[30.8794843441274...|  490.2065999848547|
|[30.9716756438877...|  494.6386097568927|
|[31.0472221394875...|  392.4973991890214|
|[31.0613251567161...|  487.5554580579016|
|[31.1239743499119...|  486.9470538397658|
|[31.2681042107507...|  423.4705331738239|
|[31.2834474760581...|  591.7810894256675|
|[31.3091926408918...|  432.7207178399336|
|[31.3123495994443...|  463.5914180279406|
|[31.3584771924370...|  495.1759504494754|
|[31.3895854806643...|  410.0696110599829|
|[31.4252268808548...|  530.7667186547619|
|[31.4459724827577...| 484.87696493512857|
|[31.4474464941278...|   418.602742095224|
|[31.5147378578019...|  489.8124879964614|
+----------

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                349|
|   mean|  503.4318992268231|
| stddev|  81.24921639156864|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean|   489.796598007511|
| stddev|  74.03743497024469|
|    min|  275.9184206503857|
|    max|  689.2356997616951|
+-------+-------------------+



In [30]:
from pyspark.ml.regression import LinearRegression

In [31]:
lm = LinearRegression(labelCol="Yearly Amount Spent")

In [32]:
model = lm.fit(train_data)

25/07/14 09:13:58 WARN Instrumentation: [de68f0a2] regParam is zero, which might cause numerical instability and overfitting.


In [33]:
import pandas as pd

In [34]:
pd.DataFrame({"Coefficients": model.coefficients}, index=coef_var)

,Coefficients
Avg Session Length,25.939579
Time on App,38.480526
Length of Membership,61.508681


In [35]:
res = model.evaluate(test_data)

In [36]:
res.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -5.638151525470505|
| -4.315347119113028|
|-13.055858324228154|
| -6.787553579620408|
| 10.383355424054571|
|-0.5946102698668483|
| 3.6493786450727157|
| -4.053709149126973|
|-14.178073696525644|
| 18.035197544856373|
| 16.783862287285785|
| -4.468151097041584|
|  7.128040606114098|
|-2.1038738779279242|
| -11.63532734100022|
|-17.400616613410023|
| -2.331049178436615|
| -10.27141858615039|
|-17.218877982553124|
|-2.4401693168724137|
+-------------------+
only showing top 20 rows



In [37]:
unlabeled_data = test_data.select("features")

In [38]:
predictions = model.transform(unlabeled_data)

In [39]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|  288.109397245385|
|[30.8364326747734...|471.81724754610264|
|[31.0662181616375...| 461.9891515319025|
|[31.1280900496166...| 564.0402403266751|
|[31.1695067987115...|416.97317537823824|
|[31.2606468698795...| 421.9212415268182|
|[31.3662121671876...| 426.9395039114122|
|[31.5171218025062...| 279.9721297995127|
|[31.5741380228732...| 558.5873458571125|
|[31.6005122003032...|461.13765394624056|
|[31.6098395733896...| 427.7616873638224|
|[31.6253601348306...| 380.8050518539658|
|[31.6548096756927...| 468.1353831214344|
|[31.7216523605090...|349.88080050980057|
|[31.8093003166791...| 548.4072267038414|
|[31.8164283341993...| 518.5231081170664|
|[31.8530748017465...| 461.6161726407886|
|[31.8648325480987...|450.16269906296407|
|[31.9048571310136...|491.16873540536926|
|[31.9120759292006...|389.97488562258013|
+--------------------+------------

In [41]:
print("MAE:", res.meanAbsoluteError)
print("MSE:", res.meanSquaredError)
print("RMSE:", res.rootMeanSquaredError)
print("R2:", res.r2)
print("Adj R2:", res.r2adj)

MAE: 8.060935081004494
MSE: 98.96560632336607
RMSE: 9.948145873647313
R2: 0.9818253000550514
Adj R2: 0.9814543878112769
